# <center>The Battle of Neighborhoods</center>

## 1. Data preparation
### 1. 1 Prepare population data
#### Total number of neighborhoods in Taipei City is 456.

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import folium
import requests
from geopy.geocoders import Nominatim
%matplotlib inline
# UTF-8 transcoded raw data
data_path = './2021_Taipei_Borough_Population_1.csv'
raw_data_df = pd.read_csv(data_path, encoding='utf_8')
# Extract the latest dataset: 2021 Feb data
raw_data_df = raw_data_df[(raw_data_df['年份']==110) & (raw_data_df['月份']==2)]
# Drop unnecessary data:
# (a) 41 year-old and above data
# (b) year, month since it is all 2021 Feb data
# (c) Total
# (d) Neighborhood population by gender; only need neighborhood population subtotal
# compose age column name to be dropped
dropped_column = list(str(i)+'歲' for i in range(41, 100))+['年份','月份', '總計', '100歲以上']
raw_data_df.drop(columns=dropped_column, axis=1, inplace=True)
raw_data_df = raw_data_df.loc[raw_data_df['性別']=='計']
raw_data_df.drop(columns=['性別'], inplace=True)
# Rename columns
raw_data_df.rename(columns={"區域代碼": "ID", "區域別": "Neighborhood"}, inplace=True)
for i in range(0, 41):
    raw_data_df.rename(columns={str(i)+'歲': 'Age-'+str(i)}, inplace=True)
# Read neighborhood annual income data
income_df = pd.read_csv('./2017_Taipei_Neighborhood_Annual_Income.csv', encoding='utf_8')

id_array = raw_data_df['ID'].values
# Organize neighborhood values
tmp_borough_id = None
tmp_borough_name = None
for i in id_array:
    if((int(math.log10(i))+1)==8):
        tmp_borough_id = i
        tmp_borough_name = raw_data_df.loc[raw_data_df['ID']==i, ['Neighborhood']].values[0,0]
    else:
        k = raw_data_df.loc[raw_data_df['ID']==i, ['Neighborhood']].values[0,0] 
        s = k + ' ' + tmp_borough_name
        raw_data_df.loc[raw_data_df['ID']==i, 'Neighborhood'] = s

raw_data_df=raw_data_df.loc[raw_data_df['ID']>64000000]
# Reset index
raw_data_df.reset_index(drop=True, inplace=True)
raw_data_df

,ID,Neighborhood,Age-0,Age-1,Age-2,Age-3,Age-4,Age-5,Age-6,Age-7,...,Age-31,Age-32,Age-33,Age-34,Age-35,Age-36,Age-37,Age-38,Age-39,Age-40
0,63000010002,莊敬里 松山區,42,47,42,45,51,54,42,41,...,63,71,68,55,62,74,79,92,101,108
1,63000010003,東榮里 松山區,33,50,43,62,78,70,80,91,...,80,80,90,68,72,99,98,110,133,120
2,63000010004,三民里 松山區,36,53,64,58,69,65,65,69,...,64,83,54,64,85,103,79,104,107,115
3,63000010005,新益里 松山區,28,36,37,26,41,39,29,33,...,56,57,50,61,67,80,86,75,83,90
4,63000010006,富錦里 松山區,35,31,48,42,50,53,80,59,...,74,66,63,58,65,77,85,83,94,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,63000120038,關渡里 北投區,77,98,96,109,129,132,124,101,...,157,184,143,142,181,221,193,234,238,239
452,63000120039,泉源里 北投區,13,11,9,18,14,14,12,16,...,33,31,30,24,28,42,41,40,36,30
453,63000120040,湖山里 北投區,8,14,8,16,5,11,9,5,...,16,16,17,31,24,24,23,26,21,24
454,63000120041,大屯里 北投區,15,9,8,11,18,18,14,13,...,15,12,9,14,25,17,29,19,32,20


### 1.2 Prepare neighborhood annual income data
#### Datasource: 2017 Taipei City Neighborhood Income Tax Statistics (Unit in k NTD$)

In [76]:
# # Read neighborhood annual income data
income_df = pd.read_csv('./2017_Taipei_Neighborhood_Annual_Income.csv', encoding='utf_8')
columns = ['City','Borough','Neighborhood','Tax unit','Income','IncomeAvg','IncomeMedian','1stQ','3rdQ', 'Std', 'CC']
income_df.columns  =columns
income_df.drop(columns=['City','Tax unit'], inplace=True)
income_df = income_df.loc[(income_df['Neighborhood']!='總計') & (income_df['Neighborhood']!='合計') & (income_df['Neighborhood']!='其他')]
income_df.sort_values(by=['Borough','IncomeMedian'], ascending=False, inplace=True)
income_df['Neighborhood'] = income_df['Neighborhood'] + ' ' + income_df['Borough']
# Merge neighborhood annual income data with population data
income_df = income_df.merge(raw_data_df, left_on='Neighborhood', right_on='Neighborhood')
income_df.drop(columns=['ID'], inplace=True)
income_df.reset_index(inplace=True, drop=True)
income_df

,Borough,Neighborhood,Income,IncomeAvg,IncomeMedian,1stQ,3rdQ,Std,CC,Age-0,...,Age-31,Age-32,Age-33,Age-34,Age-35,Age-36,Age-37,Age-38,Age-39,Age-40
0,萬華區,西門里 萬華區,1258566,1013,701,366,1221,1538.38,151.81,23,...,38,50,33,53,56,64,54,83,71,67
1,萬華區,新起里 萬華區,2507677,1112,682,374,1302,2061.48,185.38,48,...,81,100,102,83,91,135,122,119,93,106
2,萬華區,全德里 萬華區,1480791,1064,682,383,1261,1987.25,186.81,24,...,51,56,71,55,61,66,64,85,93,86
3,萬華區,壽德里 萬華區,1521674,959,679,388,1215,978.40,102.04,38,...,58,60,61,60,64,80,75,86,77,85
4,萬華區,萬壽里 萬華區,1171107,1070,676,353,1261,1306.62,122.06,23,...,32,34,27,33,44,51,44,47,55,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,中山區,行政里 中山區,2061119,996,630,351,1200,2056.10,206.40,53,...,93,97,92,85,109,90,115,100,108,110
447,中山區,新庄里 中山區,1448754,880,619,355,1134,937.62,106.53,33,...,53,79,74,59,72,88,85,74,81,89
448,中山區,正義里 中山區,1927190,1067,618,328,1086,6565.49,615.60,32,...,72,74,60,64,82,92,95,104,88,100
449,中山區,聚盛里 中山區,1354314,954,607,343,1136,1349.87,141.43,27,...,60,44,50,54,69,67,67,81,81,78


### 1.3 Query Geopy for neighborhoods' latitude and longitude

In [77]:
geolocator = Nominatim(user_agent='capstone')
# city_country = ' Taipei Taiwan'
city_country = ' 台北市 台灣'
latitude_value = []
longitude_value = []
idx_array = income_df.index.values
for i in range(0, income_df.shape[0]):
    neighborhood = income_df.iloc[i, 1] + city_country
    location = geolocator.geocode(neighborhood)
    latitude_value.append(location.latitude)
    longitude_value.append(location.longitude)
# Add geospatial data
income_df.insert(2, 'Latitude', value=latitude_value)
income_df.insert(3, 'Longitude', value=longitude_value)
income_df

,Borough,Neighborhood,Latitude,Longitude,Income,IncomeAvg,IncomeMedian,1stQ,3rdQ,Std,...,Age-31,Age-32,Age-33,Age-34,Age-35,Age-36,Age-37,Age-38,Age-39,Age-40
0,萬華區,西門里 萬華區,25.042815,121.505472,1258566,1013,701,366,1221,1538.38,...,38,50,33,53,56,64,54,83,71,67
1,萬華區,新起里 萬華區,25.041003,121.505049,2507677,1112,682,374,1302,2061.48,...,81,100,102,83,91,135,122,119,93,106
2,萬華區,全德里 萬華區,25.023361,121.498731,1480791,1064,682,383,1261,1987.25,...,51,56,71,55,61,66,64,85,93,86
3,萬華區,壽德里 萬華區,25.023295,121.500671,1521674,959,679,388,1215,978.40,...,58,60,61,60,64,80,75,86,77,85
4,萬華區,萬壽里 萬華區,25.044793,121.505332,1171107,1070,676,353,1261,1306.62,...,32,34,27,33,44,51,44,47,55,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,中山區,行政里 中山區,25.065368,121.534927,2061119,996,630,351,1200,2056.10,...,93,97,92,85,109,90,115,100,108,110
447,中山區,新庄里 中山區,25.070702,121.530619,1448754,880,619,355,1134,937.62,...,53,79,74,59,72,88,85,74,81,89
448,中山區,正義里 中山區,25.050614,121.526592,1927190,1067,618,328,1086,6565.49,...,72,74,60,64,82,92,95,104,88,100
449,中山區,聚盛里 中山區,25.059172,121.525247,1354314,954,607,343,1136,1349.87,...,60,44,50,54,69,67,67,81,81,78


### 1.4 Aggregate population of kids between 0~12 and  adults (as parents) between 25~40
#### We want to see how these two population groups affect the local neighborhood's catering business.

In [78]:
# 0 to 12 years old
kid_population = income_df.iloc[:, 11:24].sum(axis=1)
# 25 to 40 years old
parent_population = income_df.iloc[:, 36:].sum(axis=1)
income_df.insert(4, 'Kid', value=kid_population)
income_df.insert(5, 'Parent', value=parent_population)

### 1.5 Create Taipei City map and mark each neighborhood.

In [79]:
# Create a Taipei City map by using folium
taipei_location = geolocator.geocode('Taipei Taiwan')
map_taipei = folium.Map(location=[taipei_location.latitude, taipei_location.longitude], zoom_start=10)
# add markers to map
for lat, lng, neighborhood in zip(income_df['Latitude'], income_df['Longitude'], income_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_taipei)
map_taipei

### 1.6 To explore local neighborhood venue through Foursquare API

In [1]:
# Foursquare connection information
CLIENT_ID = '' 
CLIENT_SECRET = ''
VERSION = '20210323' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
RADIUS = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [85]:
# function to get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [86]:
# Get neighborhoods venues with venue categories.
taipei_venues_df = getNearbyVenues(names=income_df['Neighborhood'],
                                   latitudes=income_df['Latitude'],
                                   longitudes=income_df['Longitude']
                                  )
taipei_venues_df

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,西門里 萬華區,25.042815,121.505472,Café Dalida,25.041920,121.506372,Gay Bar
1,西門里 萬華區,25.042815,121.505472,町．記憶旅店 Cho Hotel,25.041923,121.504770,Hotel
2,西門里 萬華區,25.042815,121.505472,H&M,25.042744,121.507498,Clothing Store
3,西門里 萬華區,25.042815,121.505472,幸福堂 Xing Fu Tang,25.043385,121.507270,Bubble Tea Shop
4,西門里 萬華區,25.042815,121.505472,Tiger Sugar (老虎堂黑糖專売),25.044422,121.505484,Bubble Tea Shop
...,...,...,...,...,...,...,...
11933,聚盛里 中山區,25.059172,121.525247,全家 FamilyMart 長春店,25.054839,121.525878,Convenience Store
11934,大佳里 中山區,25.072798,121.542440,Dajia River Park (大佳河濱公園),25.074920,121.539233,Park
11935,大佳里 中山區,25.072798,121.542440,基八號水門,25.073618,121.544150,Park
11936,大佳里 中山區,25.072798,121.542440,松山機場 P.r.o. Coffee,25.072244,121.543275,Café


### 1.7 Re-organize and maintain catering related venues.

In [87]:
from sklearn.cluster import KMeans
dummy_df = pd.get_dummies(data=taipei_venues_df['Venue Category'], prefix='', prefix_sep='')
dummy_df.insert(0, column='Neighborhood', value=taipei_venues_df['Neighborhood'])
# Summarize the appearance of venue category 
dummy_df = dummy_df.groupby(by=['Neighborhood']).mean().reset_index()
dummy_df

# Drop venue categories not related to catering business
to_be_dropped = ['Airport Service',
       'Arcade', 'Art Gallery', 'Art Museum',
       'Arts & Crafts Store','Bike Trail',
       'Boarding House', 'Bookstore', 'Building',
       'Bus Station', 'Bus Stop', 'Cable Car','Clothing Store',
       'Comic Shop', 'Convenience Store',
       'Convention Center', 'Deli / Bodega',
       'Department Store', 'Electronics Store', 'Farmers Market',
       'Furniture / Home Store', 'Gaming Cafe', 'Garden', 'Golf Course',
       'Gym', 'Gym / Fitness Center', 'Historic Site', 'History Museum', 'Hospital', 
        'Hostel', 'Hotel', 'Hotel Bar', 'Indie Movie Theater', 'Intersection', 'Lake',
       'Leather Goods Store', 'Market', 'Massage Studio', 'Metro Station',
       'Miscellaneous Shop',
       'Monument / Landmark', 'Motel', 'Mountain', 'Movie Theater',
       'Moving Target', 'Multiplex', 'Museum', 'Music Venue', 'Nightclub', 'Office',
       'Organic Grocery', 'Other Great Outdoors', 'Park', 'Pastry Shop',
       'Performing Arts Venue', 'Pharmacy', 'Playground', 'Plaza', 'Pub', 'Public Art', 'Record Shop', 'Resort', 'River',
       'Rock Club', 'Scenic Lookout', 'Shoe Store',
       'Shopping Mall', 'Skating Rink', 'Soccer Field', 'Spa', 'Speakeasy',
       'Sporting Goods Shop', 'Sports Bar', 'Stadium', 'Stationery Store', 'Supermarket', 'Tea Room',
       'Tennis Court', 'Theater',
       'Theme Park', 'Theme Park Ride / Attraction', 'Tourist Information Center',
       'Toy / Game Store', 'Track Stadium', 'Trail', 'Train Station',
       'Yoga Studio', 
                'Accessories Store', 'Airport Lounge', 'Airport','Art Studio', 'Athletics & Sports', 
                'Auto Workshop', 'Baseball Field', 'Basketball Stadium', 'Baseball Stadium', 'Basketball Court', 
                'Bike Rental / Bike Share', 'Boutique', 'Bowling Alley',
                'Bridge','Buddhist Temple','Beach','Boat or Ferry',
                'Camera Store','Campground', 'Cave', 'Cha Chaan Teng','Climbing Gym',
                'College Academic Building', 'College Gym', 'College Theater',
                'Community College', 'Concert Hall','Cosmetics Shop', 'Cultural Center',
                'Cycle Studio','Discount Store', 'Dog Run',
                'Duty-free Shop', 'Fabric Shop', 'Farm', 'Fish Market', 'Film Studio', 'Flea Market','Flower Shop',
                'Fountain', 'Garden Center', 'Gift Shop','Grocery Store', 'Gym Pool','Hobby Shop',
                'Hot Spring','Laser Tag',
                'Lounge', 'Luggage Store','Mobile Phone Shop', 
                 'Motorcycle Shop','Music Store', 'Nature Preserve','National Park',
                'Optical Shop', 'Other Nightlife',
                'Paper / Office Supplies Store', 'Pedestrian Plaza', 'Pet Store', 'Pet Service',
                'Planetarium', 'Platform','Pool','Recreation Center', 'Rest Area',
                'Road', 'Roof Deck', 'Science Museum','Souvenir Shop', 'Skate Park','Sculpture Garden',
                'Temple', 'Tennis Stadium','Train', 'Tunnel',
                'Used Bookstore','Warehouse Store',"Women's Store",'Zoo Exhibit']
# to_be_dropped
dummy_df.drop(columns=to_be_dropped, inplace=True)
dummy_df.columns.values
# Aggregate total catering appearance
subtotal = dummy_df.iloc[:, 1:].sum(axis=1)
dummy_df.insert(1, column='Restaurant Density', value=subtotal)
dummy_df

,Neighborhood,Restaurant Density,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,...,Tonkatsu Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Winery,Xinjiang Restaurant,Yunnan Restaurant
0,一德里 北投區,0.250000,0.0,0.0,0.125000,0.0,0.0,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0
1,三張里 信義區,0.555556,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0
2,三愛里 中正區,0.796296,0.0,0.0,0.018519,0.0,0.0,0.00,0.037037,0.000000,...,0.0,0.0,0.0,0.018519,0.00000,0.000000,0.0,0.0,0.0,0.0
3,三民里 松山區,0.676923,0.0,0.0,0.000000,0.0,0.0,0.00,0.030769,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0
4,三犁里 信義區,0.450000,0.0,0.0,0.000000,0.0,0.0,0.05,0.050000,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,龍福里 中正區,0.678571,0.0,0.0,0.000000,0.0,0.0,0.00,0.071429,0.035714,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0
441,龍興里 中正區,0.708333,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0
442,龍門里 大安區,0.658537,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.024390,...,0.0,0.0,0.0,0.000000,0.04878,0.000000,0.0,0.0,0.0,0.0
443,龍陣里 大安區,0.757576,0.0,0.0,0.030303,0.0,0.0,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0


In [88]:
# Merge with income and population data
tmp_df = dummy_df[['Neighborhood', 'Restaurant Density']]
tmp_df = tmp_df.merge(income_df, left_on='Neighborhood', right_on='Neighborhood')
cluster_df = tmp_df[['Restaurant Density','Kid','Parent','Income','IncomeMedian']]
cluster_df

,Restaurant Density,Kid,Parent,Income,IncomeMedian
0,0.250000,865,1833,2591100,680
1,0.555556,822,1900,4468451,723
2,0.796296,1385,1571,5681976,877
3,0.676923,735,1282,3467027,795
4,0.450000,480,1132,4221331,805
...,...,...,...,...,...
440,0.678571,1079,1188,3203673,914
441,0.708333,851,870,1438881,772
442,0.658537,789,738,3035882,1080
443,0.757576,313,702,2143146,809


## 2. Data analytic
### 2.1 Clustering all neighborhood venues.

In [89]:
from sklearn.cluster import KMeans
cluster_num=20
kmeans = KMeans(n_clusters=cluster_num, random_state=0).fit(cluster_df)
kmeans.labels_

array([ 6,  8,  1,  3, 15,  3,  1,  3,  0, 11,  9, 17, 18, 11,  6, 18,  9,
       16,  6, 12,  3,  9, 10,  0, 18, 18,  3,  2, 17, 18, 12, 15,  9,  6,
        6, 17, 16,  8, 18,  3,  8,  9,  6, 17, 11, 17, 18, 19, 15, 16, 19,
        9,  2, 10, 16, 18,  3, 12,  2,  0,  6, 15,  6, 11,  9,  6, 18, 15,
        3, 14, 17,  2, 16, 18,  9,  9, 11,  7, 17,  2, 17, 16,  7,  0,  8,
        7,  9, 15,  0, 15,  8,  3,  1,  1,  0,  6,  8, 19, 19, 11, 13,  9,
       11,  2, 16, 16,  0, 17,  6,  5, 17,  9,  7, 15,  6, 15, 16,  9,  3,
        2,  1,  9, 19,  3, 17,  9, 17, 11, 11, 18, 17,  6,  6, 15, 18,  9,
        2, 16,  3, 16, 18, 11,  0,  2,  0,  5,  6,  0, 17, 11, 16,  8, 15,
        0, 10, 15, 17, 10,  9, 16,  8,  9, 17, 17,  6, 18,  3,  2,  2,  2,
       19, 18, 10, 17,  9,  0,  3,  6,  3, 19,  2, 18,  9, 18, 16,  6, 11,
       18,  2,  0,  8,  7,  6,  0,  9,  0, 11,  6,  9,  9, 11,  8, 15,  2,
        0,  3,  0,  3,  6, 12,  0,  1,  6, 17,  4,  9,  0, 11, 10,  7,  2,
       16,  0, 16, 18, 18

In [90]:
tmp_df.insert(1, column='Label', value=kmeans.labels_)

In [91]:
tmp_df

,Neighborhood,Label,Restaurant Density,Borough,Latitude,Longitude,Kid,Parent,Income,IncomeAvg,...,Age-31,Age-32,Age-33,Age-34,Age-35,Age-36,Age-37,Age-38,Age-39,Age-40
0,一德里 北投區,6,0.250000,北投區,25.132514,121.473616,865,1833,2591100,1057,...,108,130,91,102,113,111,150,137,166,155
1,三張里 信義區,8,0.555556,信義區,25.028473,121.566224,822,1900,4468451,1434,...,118,131,106,110,141,142,106,137,135,129
2,三愛里 中正區,1,0.796296,中正區,25.035841,121.530460,1385,1571,5681976,1809,...,86,84,75,86,91,111,116,127,150,140
3,三民里 松山區,3,0.676923,松山區,25.059951,121.562388,735,1282,3467027,1379,...,64,83,54,64,85,103,79,104,107,115
4,三犁里 信義區,15,0.450000,信義區,25.029591,121.572115,480,1132,4221331,2157,...,60,63,66,56,71,75,95,80,86,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,龍福里 中正區,11,0.678571,中正區,25.033115,121.515833,1079,1188,3203673,1619,...,49,56,58,53,80,94,99,110,124,118
441,龍興里 中正區,2,0.708333,中正區,25.029393,121.509785,851,870,1438881,1153,...,46,43,39,51,73,70,75,77,96,70
442,龍門里 大安區,11,0.658537,大安區,25.027949,121.536187,789,738,3035882,1936,...,31,32,46,39,43,45,51,69,87,71
443,龍陣里 大安區,18,0.757576,大安區,25.028272,121.542212,313,702,2143146,1645,...,48,51,35,39,40,37,58,42,47,59


### 2.2 Describe our data

In [92]:
compare_df = pd.DataFrame()
for i in range(cluster_num):
    label_data = tmp_df.loc[tmp_df['Label']==i][['Restaurant Density','Kid','Parent','Income','IncomeAvg','IncomeMedian','1stQ', '3rdQ', 'Std', 'CC']].mean()
    compare_df.insert(i, column='Label '+str(i), value=label_data)
target_df = compare_df.T.reset_index(drop=False)
target_df.corr(method='pearson')

,Restaurant Density,Kid,Parent,Income,IncomeAvg,IncomeMedian,1stQ,3rdQ,Std,CC
Restaurant Density,1.000000,0.746120,0.812920,0.376176,0.003599,0.723157,0.668665,0.693399,-0.231728,-0.163278
Kid,0.746120,1.000000,0.918990,0.803701,0.334932,0.906200,0.887351,0.908121,-0.008481,0.080314
Parent,0.812920,0.918990,1.000000,0.599543,0.098215,0.750502,0.729944,0.747265,-0.204512,-0.139052
Income,0.376176,0.803701,0.599543,1.000000,0.806738,0.861674,0.890751,0.891842,0.550354,0.615341
IncomeAvg,0.003599,0.334932,0.098215,0.806738,1.000000,0.561757,0.617574,0.601147,0.932782,0.955983
IncomeMedian,0.723157,0.906200,0.750502,0.861674,0.561757,1.000000,0.992968,0.996526,0.261317,0.348489
1stQ,0.668665,0.887351,0.729944,0.890751,0.617574,0.992968,1.000000,0.992745,0.326858,0.410546
3rdQ,0.693399,0.908121,0.747265,0.891842,0.601147,0.996526,0.992745,1.000000,0.300579,0.383417
Std,-0.231728,-0.008481,-0.204512,0.550354,0.932782,0.261317,0.326858,0.300579,1.000000,0.992083
CC,-0.163278,0.080314,-0.139052,0.615341,0.955983,0.348489,0.410546,0.383417,0.992083,1.000000


### 2.3 Data Modeling and Prediction
#### Use regression model to predict restaurant density of each cluster

In [93]:
# Cut Train-test group for regression
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

X_columns = ['Kid','Parent','Income','IncomeAvg','IncomeMedian','1stQ', '3rdQ', 'Std', 'CC']

# X_columns = ['Kid','Parent','Income','IncomeAvg','IncomeMedian']
y_columns = ['Restaurant Density']
X = np.asarray(target_df[X_columns])
y = np.asarray(target_df[y_columns])

print('X size: ' + str(X.shape[0]))
print('y size: ' + str(y.shape[0]))

# Normalize 
X = preprocessing.StandardScaler().fit(X).transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=6)

print('X_train size: ' + str(X_train.shape[0]))
print('y_train size: ' + str(y_train.shape[0]))
print('X_test size: ' + str(X_test.shape[0]))
print('y_test size: ' + str(X_test.shape[0]))

# reg = LinearRegression().fit(X_train, y_train)
reg = Ridge(alpha=1.0).fit(X_train, y_train)
y_hat = reg.predict(X_test)


# The coefficients
print('Coefficients: \n', reg.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_hat))
print("Mean absolute error: %.2f"
      % mean_absolute_error(y_test, y_hat))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_hat))

y_hat_all = reg.predict(X)
target_df['Predict Density'] = y_hat_all
target_df

X size: 20
y size: 20
X_train size: 14
y_train size: 14
X_test size: 6
y_test size: 6
Coefficients: 
 [[-0.00614643  0.04983129 -0.06592264 -0.0231006   0.04858638  0.0172928
   0.0395575   0.00902258  0.00345471]]
Mean squared error: 0.00
Mean absolute error: 0.02
Coefficient of determination: 0.93


,index,Restaurant Density,Kid,Parent,Income,IncomeAvg,IncomeMedian,1stQ,3rdQ,Std,CC,Predict Density
0,Label 0,0.605684,611.511628,1236.162791,2.301941e+06,1224.697674,742.581395,395.837209,1436.139535,2417.560465,182.262093,0.580758
1,Label 1,0.734764,1177.250000,1612.375000,5.484760e+06,1882.375000,923.875000,434.125000,1941.750000,6642.387500,331.871250,0.674771
2,Label 2,0.493094,435.750000,992.750000,1.356566e+06,983.000000,661.857143,375.464286,1208.035714,1135.846071,113.635714,0.516665
3,Label 3,0.639684,802.096774,1427.161290,3.456722e+06,1590.741935,820.935484,419.838710,1634.806452,5915.397419,278.193871,0.630581
4,Label 4,0.454545,617.000000,792.000000,1.085722e+07,7514.000000,912.000000,454.000000,1969.000000,205149.770000,2730.360000,0.450725
5,Label 5,0.716213,1254.250000,1764.500000,7.178550e+06,2327.750000,1072.250000,487.750000,2328.250000,6888.785000,287.722500,0.764103
6,Label 6,0.599305,705.407407,1476.925926,2.633381e+06,1206.407407,737.148148,397.277778,1407.351852,2349.164630,184.334074,0.597742
7,Label 7,0.398148,174.083333,378.500000,4.499424e+05,956.250000,585.916667,352.166667,1004.333333,2735.072500,237.024167,0.409754
8,Label 8,0.621879,982.375000,1572.625000,4.487692e+06,1724.375000,863.562500,428.312500,1738.875000,6430.672500,333.308750,0.651770
9,Label 9,0.602229,519.704545,1104.590909,1.797577e+06,1108.704545,698.068182,382.204545,1308.568182,1879.145682,159.422045,0.543929


### 2.4 Data interpretation

In [94]:
# We are interested in those labels whose restaurant density value is less than predicted value.
# This may indicate that local competition is less intense
# In other words, those neighborhoods could be defined as 'high-potential'
restaurant_df = target_df.loc[target_df['Restaurant Density'] < target_df['Predict Density']]
potential_cluster_array = restaurant_df.index.values
restaurant_df.sort_values(by='IncomeMedian', ascending=False)
# restaurant_df.iloc[0,11]

,index,Restaurant Density,Kid,Parent,Income,IncomeAvg,IncomeMedian,1stQ,3rdQ,Std,CC,Predict Density
5,Label 5,0.716213,1254.250000,1764.500000,7.178550e+06,2327.750000,1072.250000,487.750000,2328.250000,6888.785000,287.722500,0.764103
13,Label 13,0.583333,1755.000000,2179.000000,1.259633e+07,3287.000000,998.000000,470.000000,2213.000000,14630.010000,445.070000,0.639901
10,Label 10,0.633140,1084.666667,1472.166667,4.902533e+06,1894.833333,898.666667,434.916667,1847.333333,7389.060000,381.180000,0.653470
8,Label 8,0.621879,982.375000,1572.625000,4.487692e+06,1724.375000,863.562500,428.312500,1738.875000,6430.672500,333.308750,0.651770
11,Label 11,0.617425,858.193548,1603.645161,3.076780e+06,1289.677419,764.032258,402.451613,1470.129032,3555.010323,242.167419,0.618824
17,Label 17,0.509303,466.024390,1011.414634,1.582604e+06,1138.170732,695.073171,384.804878,1305.195122,2291.235366,179.025610,0.537394
16,Label 16,0.487328,351.040000,821.080000,1.125793e+06,1018.640000,675.120000,369.960000,1233.440000,1787.137600,169.524400,0.506576
2,Label 2,0.493094,435.750000,992.750000,1.356566e+06,983.000000,661.857143,375.464286,1208.035714,1135.846071,113.635714,0.516665
19,Label 19,0.370758,300.785714,677.642857,8.452646e+05,1054.285714,630.928571,369.785714,1121.571429,3014.757857,212.769286,0.469183
7,Label 7,0.398148,174.083333,378.500000,4.499424e+05,956.250000,585.916667,352.166667,1004.333333,2735.072500,237.024167,0.409754


## 3. Data visualization

### 3.1 Visualize potential neighborhoods for catering business

In [95]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# Create a potential label map by using folium
potential_df = pd.DataFrame()
# Assign a color to each cluster
label_colors_array = cm.rainbow(np.linspace(0, 1, len(potential_cluster_array)))
label_colors={}
color_idx=0
for i in potential_cluster_array:
    potential_df = potential_df.append(tmp_df.loc[tmp_df['Label'] == i])
    label_colors[i] = colors.rgb2hex(label_colors_array[color_idx])
    color_idx+=1

# Draw the map
taipei_location = geolocator.geocode('Taipei Taiwan')
map_restaurant = folium.Map(location=[taipei_location.latitude, taipei_location.longitude], zoom_start=10)

# add markers to the map
for res, lat, lon, neighborhood, cluster in zip(potential_df['Restaurant Density'], 
                                                potential_df['Latitude'], potential_df['Longitude'], 
                                                potential_df['Neighborhood'], potential_df['Label']):
    label = folium.Popup(neighborhood + '\n Cluster ' + str(cluster) + '\n Density ' + str(res), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=label_colors[cluster],
        fill=True,
        fill_color=str(label_colors[cluster]),
        fill_opacity=0.8).add_to(map_restaurant)
       
map_restaurant